In [130]:
# -*- coding: utf-8 -*-

# Traitement des TRS pour ajouter les informations de BDLexique

In [131]:
import time
import sys
import codecs
import re
import pdb # ajouter pdb.set_trace() à l'endroit où on veut le débugueur
from lxml import etree
import bs4

In [132]:
import glob
import os, fnmatch

In [133]:
parser = etree.XMLParser(remove_blank_text=True)
debug=False

## MODIFICATIONS À FAIRE :
1. changer l'organisation pour pouvoir traiter une liste de dossiers plutôt que un dossier
  - garder le lexique général en entrée
  - changer les sorties pour avoir un jeu d'exceptions par dossier
1. gérer les connecteurs
  - trouver les connecteurs multi-mots
  - tour => liste de connecteurs

## MODIFICATIONS FAITES :
1. changer l'organisation pour pouvoir traiter une liste de dossiers plutôt que un dossier
  - garder le lexique général en entrée
  - changer les sorties pour avoir un jeu d'exceptions par dossier
1. ajouter un #id aux tours et aux mots => **22/12/15**
1. gérer les parenthèses => **22/12/15**
  - les troncations
    - version longue pour BDLexique
    - version courte pour la transcription
  - les champs supplémentaires
    - mot => nbsyllabes à saisir
    - tour => raccourci
1. gérer les balises Event auto-fermantes => **23/12/15**
1. gérer les connecteurs
  - trouver les connecteurs multi-mots
  - tour => liste de connecteurs

# Préparation de l'environnement pour le script
- *dossier* doit être le répertoire où se trouvent vos fichiers (devrait finir par un /)
- *fichierTRS* contient la liste des noms de vos fichiers TRS à traiter (rempli automatiquement)
- *fichierLexique* doit être le nom du fichier BDLEXIQUE
- *fichierExceptions* doit être le nom de votre fichier INCONNUS


In [134]:
connecteurs=[
    u"et", u"alors", u"du coup", u"sinon", u"par contre", u"ça veut dire", u"enfin",
u"après", u"donc", u"puisque", u"puisqu'", u"en fait", u"mais", u"parce que", u"parce qu'", u"même si" , u"d'abord", u"et puis"
]

In [135]:
dossierCorpus="/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/"
fichierXSLT="/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS-phon.xsl"
corpusDossiers=glob.glob(dossierCorpus+"*/")
dossiersTRS={dCorpus:dCorpus+"inconnus.txt" for dCorpus in corpusDossiers}
listeDossiersTRS=corpusDossiers
sansRebalisageFichiers=["KirchnerovaAneta.trs",
                        "Xeleko-4.3.trs",
                        "Donald Trump Talks Media Coverage, Polls and His Vocal Transformation.trs",
                        "Nicolas Sarkozy et questions libres.trs",
                        "les filles à la fac 22nov.trs",
                       "Linguistique (1) (mp3cut.net) (1).trs"]

In [136]:
numPremierDossier=0

In [137]:
fichierLexique="/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/bdlexique.txt"
fichier_exceptions=True

In [138]:
voyelles=u"ieɛayøœəuoɔɑɛ̃ɔ̃ɑ̃"
voyelles=u"ieEay296@uoOòèâêûô"

Si vous n'avez pas de fichier *inconnus.txt* 
>mettez *fichier_exceptions=False* au dessus

#Modif GB 12/04/14
- mise en texte des deux blocs de traitement de la ligne de commande

In [139]:
lexicon=codecs.open(fichierLexique,"r",encoding='utf8')
bdlexique=lexicon.readlines()
lexicon.close()

In [140]:
facultatives = 0

In [141]:
phon={}
result=[]
nouvellesExceptions = []
output=[]

### Préparation des fichiers

algorithme

ajouter chaque ligne du fichier à phrases[]

In [142]:
def lowerAccents(chaine):
    return chaine.lower()

### Normaliser le mot en cours

algorithme

+ la ponctuation est remplacée par un espace
+ les espaces aux extrémités sont effacés
+ le mot est mis en minuscules

In [143]:
def trimer(mot):
    mot=lowerAccents(mot)
#    for p in u',;.:-?!()“”‘’‛‟′″´˝"«»':
    for p in u',;.:-?!“”‘’‛‟′″´˝"«»':   # Modifié le 22/12/15 pour gérer les parenthèses comme marqueur dans les mots
        mot=mot.replace(p, ' ')
    mot=mot.strip()
    return mot

In [144]:
def listerMotsCorpus(rootTRS):
    phrases=[]
    motsPhrases=[]
    elementsPhrases=[]
    motsCorpus=set()
    nPhrases=0
    for ligne in rootTRS.xpath("//Turn//text()"):
        line=ligne.strip()
        if 0: print [line]
    #    print nPhrases, line
        phrases.append(line)
#        elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ]+['’]?|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", line)
        elements=re.findall(ur"[\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ()]+['’]?|[-.…,—–()\[\]\/#\"“”‘«»<>'’=~]| ?[;!?:]", line)
        mots=[x for x in elements if not x in u"-.…,—–()\[\]\/#\"“”‘«»<>'’=~:" and not x in [u" ;",u" !",u" ?",u" :"]]
        elements=[x for x in elements if x!=u" "]
        elementsPhrases.append(elements)
        phrasePropre = u""
        for mot in mots:
            mot = trimer(mot)
            phrasePropre += mot+u" "
            m=re.search(ur"\(.*\)",mot)
            forme=mot
            graphie=mot
            if m :
                forme=re.sub(ur"\((.*)\)","\g<1>", mot)
                graphie=re.sub(ur"\(([\wâàéèêëîïôùûüçÂÀÉÈÊËÎÏÔÙÛÜÇæœÆŒ]+['’]?)\)","'", mot)
                motsAbreges[mot]={"lexical":forme, "graphie":graphie}    
            motsCorpus.add(forme)
#        phraseMots = phrasePropre.strip()
        phraseMots = phrasePropre.split()        
        motsPhrases.append(phraseMots)
        nPhrases+=1
    return (motsCorpus,motsPhrases,elementsPhrases)

algorithme

extraire de BDLex 0.forme fléchie, 1.phonétique, 2.liaison, 3.cat-gram, 4.genre+nombre

algorithme

extraire du fichier d'exceptions les mêmes données que pour BDLex

#Modif GB 12/04/14
- fait une liste des exceptions lues pour ne pas les rajouter à la fin
- éviter de tenir compte des exceptions non renseignées
 - les mots du fichier exceptions sans transcriptions étaient transcrits par une chaine vide...

## Fonctions

### Vérifier si le mot existe

algorithme

+ si le mot est dans BDLex, ok
+ s'il y a un espace dans le mot,
    * le mot est divisé en deux et
    * si les mots existent dans bdlex, ok
    * sinon les mots sont ajoutés aux nouvelles exceptions et mis entre étoiles
+ s'il y a un apostrophe dans le mot,
    * le mot est divisé en deux
    * si les mots existent dans bdlex, ok
    * sinon les mots sont ajoutés aux nouvelles exceptions et mis entre étoiles
+ dans les autres cas, le mot est ajouté aux nouvelles exceptions et mis entre étoiles

In [145]:
def verifier_mot(mot):
        sampa=""
        if mot in phon.keys():
            sampa += phon[mot][0]
        elif " " in mot:
            mots = mot.split()
            for mot in mots:
                if mot in phon.keys():
                    sampa += phon[mot][0]+" "
                elif mot != "":
                    nouvellesExceptions.append(mot)
                    sampa += "***"+mot+"*** "
        elif "'" in mot:
            mots = mot.split("'")
            mots[0]=mots[0]+"'"
            for mot in mots:
                if mot in phon.keys():
                    sampa += phon[mot][0]+" "
                elif mot != "":
                    nouvellesExceptions.append(mot)
                    sampa += "***"+mot+"*** "
        elif mot != "": 
            nouvellesExceptions.append(mot)
            sampa="***"+mot+"*** "
        return sampa

### 2. traduire le SAMPA de BDLexique en API

#Modif GB 12/04/14
- ajout du r et du â
- ajout des exemples associés en dessous

In [146]:
# traduire SAMPA-BDLex en API

def sampa2api(sampa):
    if isinstance(sampa,str):
        api=sampa.decode("utf8")
    else:
        api=sampa
    api=api.replace(u'S',u'ʃ') 
    api=api.replace(u'Z',u'ʒ')
    api=api.replace(u'N',u'ŋ')
    api=api.replace(u'J',u'ɲ')
    api=api.replace(u'r',u'ʁ') 
    api=api.replace(u'H',u'ɥ')
    api=api.replace(u'E',u'ɛ')
    api=api.replace(u'2',u'ø')
    api=api.replace(u'9',u'œ')
    api=api.replace(u'6',u'ə')
    api=api.replace(u'O',u'ɔ')
    api=api.replace(u'è',u'e')   
    api=api.replace(u'ò',u'o')    
    api=api.replace(u'â',u'ɑ̃')   
    api=api.replace(u'ê',u'ɛ̃')   
    api=api.replace(u'û',u'œ̃')  
    api=api.replace(u'ô',u'ɔ̃')       
    api=api.replace(u'@',u'ə')
    api=api.replace(u'n"',u'n') 
    api=api.replace(u't"',u't') 
    api=api.replace(u'z"',u'z') 
    api=api.replace(u'R"',u'ʁ') 
    api=api.replace(u'p"',u'p') 
    return api

### 4. Vérifier si la liaison est possible

algorithme

+ si le mot courant et le suivant ne sont pas dans lexicon, pas de liaison
+ si le mot a une consonne dans le champ de la voyelle de liaison, check1 est vrai
+ si le mot suivant commence par une voyelle, check2 est vrai

  si check1 et check2 sont vrais, il y a liaison

In [147]:
def liaison_possible(phrase ,mot , mot_numero):
    check1=0
    check2=0
    if mot in phon and len(phrase)>mot_numero+1 and phrase[mot_numero+1] in phon:
        consonnes=['k"', '(kt)"', 'n"', 'p"', 'R"', '@t"', 't"', '-V', '+V', '@z"', 'z"']
        phoneme=phon[mot][2]
        for phoneme in consonnes:
            check1=1
        
        voyelles=["H", "j", "w", "E", "a", "2", "9", "6", "@", "y", "u", "O", u"ò", "o", "e", u"è", u"ê", u"û", u"ô", "i"]
        mot_suivant=phon[phrase[mot_numero+1]][1]
        for v in voyelles:
            if mot_suivant.startswith(v):
                check2=1

    if check1 and check2 :
        return True
    else:
        return False

### 5. vérifier si la liaison est obligatoire

algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [148]:
def liaison_obligatoire(phrase, mot, mot_numero):
    determinant=["d", "P"]
    nom=["N", "G", "M"]
    adjectif=["J", "G", "M"]
    pronompers=["P"]
    verbe=["V"]
    catgram_mot1=phon[phrase[mot_numero]][3]
    catgram_mot2=phon[phrase[mot_numero+1]][3]

    if catgram_mot1 in determinant and catgram_mot2 in nom :
        return True

    elif catgram_mot1 in determinant and catgram_mot2 in adjectif :
        return True
 
    elif catgram_mot1 in pronompers and catgram_mot2 in verbe :
        return True

    elif catgram_mot1 in verbe and catgram_mot2 in pronompers :
        return True

    else:
        return False

Cas de figure possibles:

- DET + N
    * ri + N:   d'animal, 
    * di + N:   certains éléphants
    * rd + N:   les animaux
    * dd + N:   ces étés, cet été
    * dp + N:   ton anorak
    * rc + N:   aux armes
- DET + ADJ:
    * ri + ADJ:   d'énormes
    * di + ADJ:   plusieurs immenses
    * rd + ADJ:   les immenses
    * dd + ADJ:   cet immense
    * dp + ADJ:   son immense
    * rc + ADJ:   aux immenses
- PERS + V:
    * SS + V:   m'épate
- V + PRO PERS: 
    * V + SS:   vont-ils


algorithme

+ si le mot courant et le suivant sont dans un des cas de figure, il y a liaison
+ sinon pas de liaison

In [149]:
# vérifier si la liaison est facultative
def liaison_facultative(phrase, mot, mot_numero):
    #pdb.set_trace()
    nom=["N", "G", "M"]
    pluriel=["MP", "FP"]
    adjectif=["J", "G", "M"]
    verbe=["V"]
    pronompers=["P"]
    adverbe=["A"]
    preposition=["p"]
    catgram_mot1=phon[phrase[mot_numero]][3]
    catgram_mot2=phon[phrase[mot_numero+1]][3]
    genre_mot1=phon[phrase[mot_numero]][4]
    
    if (catgram_mot1 in nom) and (phon[phrase[mot_numero]][4] in pluriel) and (catgram_mot2 in adjectif) : 
        return True

    elif (catgram_mot1 in verbe) and (catgram_mot2 not in pronompers):
        return True

    elif catgram_mot1 in adverbe :
        return True
    
    elif catgram_mot1 in preposition : 
        return True

    else :
        return False

Cas de figure possibles :

- N pl + ADJ: 
    * N + ADJ: monstres énormes 
    * G + ADJ: rivaux énormes
- VERBE + TOUT-SAUF-PRO-PERS:
    * V + N sont éléphants
    * V + G sommes abdicaires
    * V + V sommes assis
    * V + A sommes admirablement
    * V + p sommes autour de
    * V + di ont aucune
    * V + rc sommes au
- ADV + QQCH:
    * ADV + N vraiment abruti
    * ADV + G vraiment abandonné
    * ADV + V vraiment aimé
    * ADV + J vraiment étonnant
    * ADV + ss vraiment ils
    * ADV + A vraiment étonnamment
    * ADV + p vraiment attendu
    * ADV + di vraiment autre 
    * ADV + rc vraiment au
- PREP + QQCH:
    * PREP + N très amoureux
    * PREP + G très abandonné
    * PREP + V très aimé
    * PREP + J très étonnant
    * PREP + SS très ils
    * PREP + A très étonnamment
    * PREP + p très attendu
    * PREP + di très autre
    * PREP + rc très au


## Traitement

+ Partie 1
*chaque phrase est prise individuellement,
    * découpée en blocs,
        * qui sont chacuns trimés si ce sont des mots
        * s'il y a plusieurs mots dans le bloc, ils sont séparés
    + Partie 2
    * pour chaque couple de mots
        * si la liaison est possible,
            * et qu'elle est obligatoire, l'api avec la liaison est généré
            * et qu'elle est facultative,
                * si l'utilisateur l'a choisi, l'api avec la liaison est généré
                * sinon l'api sans la liaison est généré

        + Partie 3
        * si la liaison n'est pas possible,
            * si le mot est dans bdlex, l'api est généré
            * sinon le mot est laissé tel quel (il a déjà les étoiles)        

    * pour le dernier mot de la phrase, 
        * si le mot est dans bdlex, l'api est généré
        * sinon le mot est laissé tel quel (il a déjà les étoiles) 

+ Partie 4
* le message à l'utilisateur et la phrase en api est imprimée

#Modif GB 12/04/14
- suppression du délai dans la boucle
 - pour 1500 lignes => 3 secondes sans ralentisseur, 1503 secondes avec 

In [150]:
from lxml.builder import E

In [151]:
def compterVoyelles(chaine):
    result=0
    for element in chaine:
        if element in voyelles:
            result+=1
    return result

#Début de l'enchassement en XML (7/12/15)
- récupérer la ponctuation et les sauts de lignes pour rendre le texte lisible
- ajouter le reste des informations du lexique dans la balise

In [152]:
def enchasseBDLexique(nphrase,nmot,liaison=False):
    boolAbrege=False
    motTRS=motsPhrases[nphrase][nmot]
    if motTRS in motsAbreges:
        mot=motsAbreges[motTRS]["lexical"]
        graphie=motsAbreges[motTRS]["graphie"]
        boolAbrege=True
    else:
        mot=motTRS
        graphie=motTRS
#    print mot
    if mot in phon: 
        phono=sampa2api(phon[mot][1])
        if liaison:
            phono+=sampa2api(phon[mot][2])
        cat=phon[mot][3]
        if cat in [u"J",u"K"]:
            cat=u"Adj"
        ms=phon[mot][4]
        vs=phon[mot][5]
        lexeme=phon[mot][6].upper()
        freq=phon[mot][8]
        nbVoyelles=str(compterVoyelles(phon[mot][1]))
        if u" " in vs:
            vs=u""
    else:
        phono=verifier_mot(mot)[:-1]
        cat=u"???"
        ms=""
        vs=""
        lexeme="???"
        freq=""
        nbVoyelles=""
    motAttributs={"cat":cat,"ms":ms,"vs":vs,"phon":phono,"nbsyll":nbVoyelles, "lexeme":lexeme, "freq":freq, "id":"%05d%03d"%(nphrase,nmot)}
    if boolAbrege:
        motAttributs["ABnbsyll"]=""
        motAttributs["ABphon"]=""
    result=E.motBDL(graphie,motAttributs)
#    print etree.tostring(result,encoding="utf8")
#    print (cat,ms,vs,phono,mot)
#    u'<mot cat="%s" ms="%s" vs="%s" phon="%s">%s</mot>' % (cat,ms,vs,phono,mot)
    return result
    
def enchasseXML(mot, phono):
    if isinstance(phono,str):
        phono=phono.decode("utf8")
    result=E.motBDL(mot,{"phon":phono})
#    u'<mot phon="%s">%s</mot>' % (phono, mot)
    return result

def enchasseTour(phrase):
    result=E.tour(phrase,{"id":"%06d"%nPhrase})
    return result

def enchasseNonMot(nonmot):
    result=E.punct(nonmot)
#    u'<punct>%s</punct>' % (nonmot)
    return result

In [153]:
def traitementTRS(rootTRS):
    a=1
    nPhrase=0
    for ligne in rootTRS.xpath("//Turn//text()"):
        phrase=ligne.strip()
        api=E.tour()
        mot_numero=0
        element_numero=0
        while elementsPhrases[nPhrase] and element_numero < len(elementsPhrases[nPhrase]):
            if not mot_numero < len(motsPhrases[nPhrase]) or motsPhrases[nPhrase][mot_numero]!=elementsPhrases[nPhrase][element_numero].lower():
                api.append(enchasseNonMot(elementsPhrases[nPhrase][element_numero]))
            elif liaison_possible(motsPhrases[nPhrase], motsPhrases[nPhrase][mot_numero], mot_numero):
                if liaison_obligatoire(motsPhrases[nPhrase], motsPhrases[nPhrase][mot_numero], mot_numero):
                    api.append(enchasseBDLexique(nPhrase,mot_numero,True))
                elif liaison_facultative(motsPhrases[nPhrase], motsPhrases[nPhrase][mot_numero], mot_numero):
                    if facultatives:
                        api.append(enchasseBDLexique(nPhrase,mot_numero,True))
                    else :
                        api.append(enchasseBDLexique(nPhrase,mot_numero))
                else:
                    api.append(enchasseBDLexique(nPhrase,mot_numero))
                mot_numero+=1
            else:
                api.append(enchasseBDLexique(nPhrase,mot_numero))
                mot_numero = mot_numero+1
            element_numero+=1
        a=a+1
        if phrase!="":
            phraseConnecteurs=set()
            for connecteur in connecteurs:
                if " " in connecteur:
                    connecteurParties=connecteur.split(" ")
                else:
                    connecteurParties=[connecteur]
                for i in range(len(elementsPhrases[nPhrase])-len(connecteurParties)+1):
                    if connecteurParties==elementsPhrases[nPhrase][i:i+len(connecteurParties)]:
                        phraseConnecteurs.add(connecteur)
            if phraseConnecteurs:
#                print phraseConnecteurs
                api.set("connecteurs",",".join(phraseConnecteurs))
            api.set("nbmots",str(len(api.xpath("//tour/motBDL"))))
            api.set("id","%06d"%nPhrase)
            noeudAttachement=ligne.getparent()
            if noeudAttachement.text==None:
                noeudAttachement.tail=None
                noeudAttachement.addnext(api)
            else:
                noeudAttachement.text=None
                try:
                    noeudAttachement.append(api)
                except TypeError:
                    print phrase, noeudAttachement
                    noeudAttachement.append(api)
        else:
            ligne.getparent().tail=None
        nPhrase+=1

#Modif GB 12/04/14
- Insertion d'un set sur les nouvellesExceptions pour éviter les entrées multiples
- Ajout d'un test pour vérifier que les nouvellesExceptions sont nouvelles

#TO DO
- Ajouter un message pour dire que le résultat a été concaténé au fichier existant si c'est le cas.

In [154]:
def extraireMotsTRS(motsCorpus,phon):
    for entry in bdlexique:
        entry=entry.strip()
        p=entry.split(u';')
        if p[0].lower() in motsCorpus:
            if p[2]=="@" and not p[3] in ["N","V","J","K"]:
                p[1]+=p[2]
                p[2]=""
                if len(p)<7:
                    for i in range(len(p)+1,7):
                        p.append("")
            phon[p[0].lower()]=(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8],p[9])
    return phon


In [155]:
#1.2.b. mettre les phrases phonémisées dans un fichier
enteteXML=[
            u'<?xml version="1.0" encoding="UTF8" standalone="yes"?>',
            u'<?xml-stylesheet type="text/xsl" href="phonemise-TRS.xsl"?>',
            u'<!DOCTYPE Trans SYSTEM "trans-14-corpus.dtd">'
          ]

#print [etree.tostring(rootTRS,pretty_print=True,encoding="utf8").decode("utf8")]
motsAbreges={}

In [156]:
def baliserTRS(nomTRS):
    with open(nomTRS,"r") as temp:
        header= temp.readlines()[0]
        s=re.search(ur'encoding="(.+)"',header)
        if s:
            TRS=codecs.open(nomTRS,"r",encoding=s.group(1)).readlines()
        else:
            TRS=open(nomTRS,"r").readlines()
    sortie=""
    fins=[]
    debs=[]
    for numLigne,ligne in enumerate(TRS[2:]):
        ligne=ligne.strip()
        if 0<=numLigne <=10:
            print ligne
        disfluenceGen=re.match('<Event desc="disflu" type="(noise|lexical|pronounce|language|entities)" extent="(begin|end)"/>',ligne)
        disfluenceSpec=re.match('<Event desc="([Mm]d|[Rr]ep|[Aa]uto[Cc]|[Nn]on[Ff]inie|[Mm][Cc]oup)" type="pronounce" extent="(begin|end)"/>',ligne)        
        eventAutre=re.match('<Event desc="([^"]+)" type="([^"]+)" extent="(begin|end)"/>',ligne)        
        tagTurn=re.match('<(/?)Turn.*>',ligne)        
        if disfluenceGen:
            if debug: print "disfluGen",disfluenceGen.group(2)
            if disfluenceGen.group(2)=="begin":
                sortie+=(ligne+"\n")
                sortie+=('<disfluence type="%s">'%disfluenceGen.group(1)+"\n")
                fins.append("</disfluence>")
            elif disfluenceGen.group(2)=="end":
                sortie+=("</disfluence>"+"\n")
                chaine=fins.pop()
                if chaine!="</disfluence>":
                    print "PB",chaine, "</disfluence>", numLigne
                sortie+=(ligne+"\n")
        elif disfluenceSpec:
            if debug: print "disfluSpec",disfluenceSpec.group(2)
            if disfluenceSpec.group(2)=="begin":
                sortie+=(ligne+"\n")
                sortie+=('<disfluence type="%s">'%disfluenceSpec.group(1)+"\n")
                fins.append("</disfluence>")
            elif disfluenceSpec.group(2)=="end":
                sortie+=("</disfluence>"+"\n")
                chaine=fins.pop()
                if chaine!="</disfluence>":
                    print "PB",chaine, "</disfluence>", numLigne
                sortie+=(ligne+"\n")
        elif eventAutre:
            if debug: print "Autre",eventAutre.group(3)
            descEvent="".join([l if not l in " *" else "_" for l in eventAutre.group(1)])
            typeEvent="".join([l if not l in " *" else "_" for l in eventAutre.group(2)])
            if eventAutre.group(3)=="begin":
                sortie+=(ligne+"\n")
                sortie+=('<%s desc="%s">'%(typeEvent,descEvent)+"\n")
                fins.append("</%s>"%typeEvent)
            elif eventAutre.group(3)=="end":
#                print numLigne
                if fins:
                    sortie+=("</%s>"%typeEvent+"\n")
                    chaine=fins.pop()
                else:
                    print "PB no stack to pop", typeEvent,numLigne
                if chaine!="</%s>"%typeEvent:
                    print "PB",chaine, typeEvent,numLigne
                sortie+=(ligne+"\n")
        elif tagTurn:
            if debug: print tagTurn.group(1)+"Turn"
            if tagTurn.group(1)=="/" and fins:
                lenFins=len(fins)
                for num in range(lenFins):
                    chaine=fins.pop()
                    sortie+=(chaine+"\n")
                    debs.append(chaine.replace("/",""))
            sortie+=(ligne+"\n")
            if tagTurn.group(1)=="" and debs:
                lenDebs=len(debs)
                for num in range(lenDebs):
                    chaine=debs.pop()
                    sortie+=(chaine+"\n")
                    fins.append(chaine.replace("<","</"))
        else:
            sortie+=(ligne+"\n")
        if debug and (debs or fins):
            print debs, fins
    return sortie

In [157]:
dossiersTRS

{'/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ALEXIA CLEMENT_11603/': '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ALEXIA CLEMENT_11603/inconnus.txt',
 '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANETA KIRCHNEROVA_11613/': '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANETA KIRCHNEROVA_11613/inconnus.txt',
 '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANGELIQUE BECKER_11639/': '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANGELIQUE BECKER_11639/inconnus.txt',
 '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANGELIQUE BOKOLOMBE_11643/': '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANGELIQUE BOKOLOMBE_11643/inconnus.txt',
 '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANNE-SOPHIE DACKO_11566/': '/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANNE-SOPHIE DACKO_11566/inconnus.txt',
 '/Users/gilles/own

In [162]:
%%debug
paires={num:el for num,el in enumerate(listeDossiersTRS)}
for paire in paires:
    print paire,paires[paire]
#debugging

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()

ipdb> n
u'g\xe9nant'
ipdb> n
u'g\xe9nant'
ipdb> n
u'g\xe9nant'
ipdb> n
u'g\xe9nant'
ipdb> n
u'g\xe9nant'
ipdb> n
u'g\xe9nant'
ipdb> c
0 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ALEXIA CLEMENT_11603/
1 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANETA KIRCHNEROVA_11613/
2 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANGELIQUE BECKER_11639/
3 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANGELIQUE BOKOLOMBE_11643/
4 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ANNE-SOPHIE DACKO_11566/
5 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/AXELLE DE OLIVEIRA_11655/
6 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/BORIS DUBUC_11572/
7 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/CAMILLE LANDAIS_11632/
8 /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/

In [159]:
oldExceptions=[]
for dossier in listeDossiersTRS[numPremierDossier:]:
    print dossier
    fichiersTRS=glob.glob(dossier+"/*.trs")
    fichiersTRS=[f for f in fichiersTRS if not f.endswith("-BDL2.trs")]
    fichierExceptions=dossiersTRS[dossier]
    print "fichier Exception",fichierExceptions
    print
    nouvellesExceptions=[]
    boolExceptions=True
    try:
#        print fichierExceptions
        exceptions=codecs.open(fichierExceptions,"r",encoding='utf8')
        inconnus=exceptions.readlines()
        exceptions.close()
#        print inconnus
    except IOError:
        boolExceptions=False
    if fichier_exceptions and boolExceptions:
        oldExceptions=[]
        for entry in inconnus:
            entry=entry.strip()
            if 0: print entry
            p=entry.split(";")
            if len(p)<9:
                for i in range(len(p)+1,10):
                    p.append(u"")
            if 0: print p
            if len(p[1])!=0:
                phon[p[0].lower()]=(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8])
            oldExceptions.append(p[0].lower())
    for numTRS,nomTRS in enumerate(fichiersTRS):
        print nomTRS
        fichierBDL=nomTRS[:-4]+"-BDL2.xml"
        if nomTRS.split("/")[-1] in sansRebalisageFichiers:
            print "SANS reBALISER"
            xmlTRS=etree.parse(nomTRS,parser)
            print "FIN parse"
        else:
            fichierTRS=baliserTRS(nomTRS)
            print "FIN reBALISER"
            xmlTRS=etree.fromstring(fichierTRS,parser)
            print "FIN fromstring"
        (motsCorpus,motsPhrases,elementsPhrases)=listerMotsCorpus(xmlTRS)
        print "FIN lister mots"
        phon=extraireMotsTRS(motsCorpus,phon)
        print "FIN extraire mots"
        traitementTRS(xmlTRS)
        print "FIN traitement"
        with codecs.open(fichierBDL, "w", encoding='utf8') as f:
            for ligne in enteteXML:
                f.write(ligne+u"\n")
            f.write(etree.tostring(xmlTRS,pretty_print=True,encoding="utf8").decode("utf8"))
        cliBDL=fichierBDL.replace(" ","\ ")
        cliXSLT=fichierXSLT.replace(" ","\ ")
        cliPhonTRS=fichierBDL.replace(".xml",".trs").replace(" ","\ ")
        cliText="xsltproc %s %s > %s"%(cliXSLT,cliBDL,cliPhonTRS)
        print cliText
        os.system(cliText)  
    with codecs.open(fichierExceptions, "a", encoding='utf8') as f:
        print "fichierExceptions",fichierExceptions
        for n in set(nouvellesExceptions):
            print n,
            if not (n in oldExceptions): 
                f.write(n+u";;;;;;;;;;;;")
                f.write("\n")
        print
    oldExceptions=[]


/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ALEXIA CLEMENT_11603/
fichier Exception /Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ALEXIA CLEMENT_11603/inconnus.txt

/Users/gilles/ownCloud/Cours/Bordeaux/L2-XML/XML-Ressources/TRS/ALEXIA CLEMENT_11603/67 Ans Jean François, 68 Ans Marie-jeanne Et 53 Ans Celine.trs
<Trans scribe="Propriétaire" audio_filename="67 Ans Jean François, 68 Ans Marie-jeanne Et 53 Ans Celine" version="3" version_date="161201">
<Speakers>
<Speaker id="spk1" name="Jean-François" check="no" type="male" dialect="native" accent="" scope="local"/>
<Speaker id="spk2" name="Céline" check="no" type="female" dialect="native" accent="" scope="local"/>
<Speaker id="spk3" name="Marie-Jeanne " check="no" type="female" dialect="native" accent="" scope="local"/>
</Speakers>
<Episode>
<Section type="report" startTime="0" endTime="452.833">
<Turn startTime="0" endTime="12.177" speaker="spk1">
<Sync time="0"/>
Je suis d'accord avec toi c'est tr

In [160]:
#print fichierTRS